In [1]:
5+5

10

In [2]:
import os
os.chdir("../")
%pwd

'd:\\youtube\\machine-learning-llm'

## Entity (/entity/config_entity) 

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class Vgg16TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

## Config Manager (/config/config_manager) 

In [4]:
from llm.logchain_model.constants import * 
from llm.logchain_model.utils.common import read_yaml, create_directories
import tensorflow as tf 

In [5]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_vgg16_training_config(self) -> Vgg16TrainingConfig:
        vgg16_training_model = self.config.vgg16_training_model
        vgg16_base_model = self.config.vgg16_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([
            Path(vgg16_training_model.root_dir)
        ])

        
        training_config = Vgg16TrainingConfig(
            root_dir=Path(vgg16_training_model.root_dir),
            trained_model_path=Path(vgg16_training_model.trained_model_path),
            updated_base_model_path=Path(vgg16_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


## component build (/component/vgg16_train_model.py)

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
class Vgg16ModelTraining:
    def __init__(self, config: Vgg16TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator


        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )



    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        print('train -- 01--')
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )
        print('train -- 02--')
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )



In [14]:
from IPython.display import display, Image

In [16]:

from IPython.display import display, Image

try:
    config = ConfigManager()
    training_config = config.get_vgg16_training_config()
    
    vggtraining = Vgg16ModelTraining(config=training_config)
    vggtraining.get_base_model()
    vggtraining.train_valid_generator()
    # Issue - ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.
    vggtraining.train()
    
except Exception as e:
    raise e

[2023-12-27 12:18:40,661: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-27 12:18:40,665: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-27 12:18:40,667: INFO: common: created directory at: artifacts]
[2023-12-27 12:18:40,668: INFO: common: created directory at: artifacts\training_model]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
train -- 01--


ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.